In [1]:
import json
import torch
# import openai
import pandas as pd
from tqdm import tqdm
import time
# from dotenv import load_dotenv
import os
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

c:\Users\hariz\anaconda3\envs\base_dl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_tweets(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        tweets = json.load(f)
    return tweets

In [3]:
# tweets = [
#     {
#         "tweet_count": 1,
#         "unique_id": "azwar6102",
#         "username": "Azwar Rozali",
#         "text": "Sebenarnya presiden Prabowo tidak perlu banyak bicara, yang dibutuhkan rakyat aksi nyata. Menaikkan PPN, menaikkan pajak kendaraan bermotor, menaikkan gaji honorer Rp 500ribu tidak perlu 100 hari. Kenapa kasus-kasus lain diabaikan, pengesahan RUU rampas aset presiden diam.",
#         "hashtags": [],
#         "created_at": "2025-01-01 03:26:45",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 2,
#         "unique_id": "B_dzoel",
#         "username": "°Bang_Dzoel°",
#         "text": "Hadiah 100 hari kerja kabinet Merah-Putih @prabowo\n\n1. PPN 12%\n2. Daun kelor pengganti Susu\n3. Judol Peringkat Tertinggi Dunia\n4. Memaafkan Koruptor\n5.\n6.\n7.\n\nMenuju Indonesia Emas\nJayalah Indonesiaku\n\n#100HariKabinetOmon \n\n. https://t.co/AIvUpH6PSu",
#         "hashtags": [
#             "100HariKabinetOmon"
#         ],
#         "created_at": "2025-01-01 01:34:19",
#         "retweet_count": 12,
#         "favorite_count": 28,
#         "reply_count": 5,
#         "image_paths": [
#             "C:\\Users\\hariz\\projects\\tweet_scraping\\images\\media_2_0.jpg"
#         ]
#     },
#     {
#         "tweet_count": 3,
#         "unique_id": "RiskyDeswandi",
#         "username": "riskydeswandi28",
#         "text": "H-19, penilaian 100 Hari Kerja Pemerintahan Republik Indonesia Kabinet Merah Putih.\n#prabowo",
#         "hashtags": [
#             "prabowo"
#         ],
#         "created_at": "2025-01-01 00:17:07",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 4,
#         "unique_id": "AlimyFikri",
#         "username": "Singgih",
#         "text": "@Dennysiregar7 Baru belum 100 hari sdh bosan sama Prabowo bang @Dennysiregar7 , apa minta diganti bang?",
#         "hashtags": [],
#         "created_at": "2025-01-01 13:15:48",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 5,
#         "unique_id": "DitriRizki_D3",
#         "username": "dit_RI",
#         "text": "@Mdy_Asmara1701 Blunder Pak Prabowo kok bisa milih dia jadi menteri… nilai 11/100 lagi buat pak Prabowo menuju masa habis 100 hari kerja pertama rezim Prabowo…  mau menangis kok Indonesia dibuat main2 sama penguasa, pendukung &amp; oligarki…",
#         "hashtags": [],
#         "created_at": "2025-01-01 23:56:41",
#         "retweet_count": 0,
#         "favorite_count": 9,
#         "reply_count": 1,
#         "image_paths": []
#     }
# ]

In [4]:
# ChatGPT openAI api
# def analyze_sentiment(text):
#     """
#     Analyze the sentiment of the given text using the OpenAI API.
#     The result is not good, for now I'll not use this.
#     """
#     try:
#         response = openai.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": "You are a sentiment analyzer for Bahasa Indonesia. Classify the following text as POSITIVE, NEGATIVE, or NEUTRAL. Only respond with one of these three words."},
#                 {"role": "user", "content": text}
#             ],
#             temperature=0
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Error analyzing sentiment: {e}")
#         time.sleep(20)  # Wait if we hit rate limits
#         return "NEUTRAL"  # Default fallback

In [5]:
# IndoRoBERTa-Base-Sentiment-Indonesian-Social-Media
# Load the base tokenizer
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Load the base model
# base_model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

sentiment_model = AutoModelForSequenceClassification.from_pretrained(
    "yogie27/IndoRoBERTa-Base-Sentiment-Indonesian-Social-Media"
)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_mapping[predicted_class]


In [7]:
# Load tweets
tweets = load_tweets('tweets.json')

# Create a list to store results
results = []

# Process each tweet
for tweet in tqdm(tweets, desc="Analyzing tweets"):
    sentiment = analyze_sentiment(tweet['text'])
    tweet['sentiment'] = sentiment  # Add sentiment directly to tweet object
    time.sleep(0.5)

# Save updated tweets back to the original file
with open('tweets.json', 'w', encoding='utf-8') as f:
    json.dump(tweets, f, ensure_ascii=False, indent=4)

print("Tweets updated with sentiment analysis")

    

Analyzing tweets:   3%|▎         | 93/3696 [00:55<35:49,  1.68it/s]


IndexError: index out of range in self